In [ ]:
import os
import cv2
import numpy as np
import pandas as pd



from tensorflow import keras
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from matplotlib import pyplot
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
import seaborn as sns
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.models import model_from_json
from keras.preprocessing import image
from tensorflow.keras import optimizers
from tensorflow.keras.applications import Xception

In [ ]:
def load_subdata(data):
    X1 = []
    y1 = []
    size = 229
    lbl = 0
    dic = {}
    for folder_name in os.listdir(data):
        
        Label = lbl
        dic[folder_name] = Label  
        count_imag = 0;

        for filename in os.listdir(data + '/' + folder_name):
            image = tf.keras.preprocessing.image.load_img(data +'/'+ folder_name + '/' + filename,color_mode='rgb', target_size=(size,size))
            arr = keras.preprocessing.image.img_to_array(image)
            X1.append(np.array([arr]))
            y1.append(Label) 
        lbl +=1   
         
    X1 = np.asarray(X1).reshape(len(X1),229,229,3).astype(np.uint8)
    y1 = np.asarray(y1).astype(np.uint8)
    return X1,y1, dic

In [ ]:
X_train, y_train, label_dic = load_subdata('D:/train')
X_test, y_test, test_label_dic = load_subdata('D:/test')    
X_val, y_val, val_label_dic = load_subdata('D:/val')

In [ ]:
print("Training Images Shape (x train shape) :", X_train.shape)
print("Label of training images (y train shape) :",y_train.shape)

print("Validation Images Shape (x val shape) :",X_val.shape)
print("Label of Validation images (y val shape) :",y_val.shape)

print("Test Images Shape (x val shape) :",X_test.shape)
print("Label of Test images (y val shape) :",y_test.shape)

In [5]:
unique, counts = np.unique(y_train, return_counts=True)
label_count = dict(zip(unique, counts))

In [6]:
y_trainHot = np.uint8(to_categorical(y_train, num_classes = 93))
y_testHot = np.uint8(to_categorical(y_test, num_classes = 93))
y_valHot = np.uint8(to_categorical(y_val, num_classes = 93))

In [7]:
def shuffle(a, b):
    shuffled_a = np.empty(a.shape, dtype=np.uint8)
    shuffled_b = np.empty(b.shape, dtype=np.uint8)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [8]:
X_train_s, y_train_s = shuffle(X_train, y_trainHot)
X_test_s, y_test_s = shuffle(X_test, y_testHot)
X_val_s, y_val_s = shuffle(X_val, y_valHot)

In [9]:
print("One hot encoded labels")
print(y_train_s.shape)
print(y_test_s.shape)
print(y_val_s.shape)

One hot encoded labels
(6391, 93)
(887, 93)
(762, 93)


In [ ]:
plt.figure(figsize=(10,10))
plt.title('Random Image from Dataset')
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_val_s[i])
    
plt.show()

In [ ]:
def Xception_Net():
    base_model =tf.keras.applications.Xception(input_shape = (229, 229, 3), # Shape of our images
                                 include_top = False, 
                                 weights = 'imagenet',pooling='avg',classes=93)
    base_model.trainable = True 
    base_model.layers[0].trainable=False
    prediction_layer = Dense(units=93, activation='softmax') 
    model = Sequential()
    model.add(base_model)
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(prediction_layer)
    return model

In [ ]:
xcep_model=Xception_Net()

In [ ]:
xcep_model.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.0001),
              metrics = ['accuracy']) #when the learning rate was set to 0.01, the model showed a poor performance comparated to lr=0.0001
xcep_model.summary()

In [ ]:
hist =xcep_model.fit(X_train_s, y_train_s, epochs=10,batch_size=64,validation_data=(X_val_s,y_val_s))

In [ ]:
def plot_model(history):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,4))
    fig.suptitle('Model Accuracy and Loss')

    ax1.plot(hist.history['accuracy'])
    ax1.plot(hist.history['val_accuracy'])
    ax1.title.set_text('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train','Valid'],loc=4)

    ax2.plot(hist.history['loss'])
    ax2.plot(hist.history['val_loss'])
    ax2.title.set_text('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train','Valid'],loc=1)

    fig.show()
    
plot_model(hist)


In [ ]:
def plot_cf_mat(cf_matrix):
    
    classes = list(label_dic.keys())
    plt.rcParams["figure.figsize"] = (10,7)
    ax = plt.axes()
    sns.heatmap(cf_matrix, annot=True, 
              annot_kws={"size": 10}, 
              xticklabels=classes, 
              yticklabels=classes, ax = ax,fmt="d")
    ax.set_title('Confusion matrix')
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    plt.show()

In [ ]:
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test_s)
    y_test = y_test_s
    Y_pred = np.argmax(y_pred, axis=1)
    Y_Test = np.argmax(y_test, axis=1)
    
    class_names = list(label_dic.keys())
    print("Classification Report:")
    
    # Print the evaluation metrics for the dataset.
    print(classification_report(Y_Test, Y_pred, target_names=class_names))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(Y_Test, Y_pred)
    
    accuracy = recall_score(Y_Test, Y_pred, average=None)
    precision=precision_score(Y_Test,Y_pred,average=None)

    #class_names = class_names
    class_names = list(label_dic.keys())

    print("Confusion Matrix:")

    print(cf_matrix)
    plot_cf_mat(cf_matrix)
    ## Display the visualization of the Confusion Matrix.
    plt.show()
    
    print("Accuracy for each class: ")
    
    for i in range(len(accuracy)):
        print(f"{class_names[i]}:  {format(accuracy[i]*100, '.2f')}%")
    return accuracy,precision

In [ ]:
accuracy_xcep, precision_xcep = model_Evaluate(xcep_model)

In [ ]:
Xception_Model = xcep_model.to_json()
with open("xcep_model.json", "w") as json_file:
    json_file.write(Xception_Model)
# serialize weights to HDF5
xcep_model.save_weights("xcep_model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json
json_file = open('xcep_model.json', 'r')
Xception_Model_json = json_file.read()
json_file.close()
Xception_Model = model_from_json(Xception_Model_json)
# load weights into new model
Xception_Model.load_weights("xcep_model.h5")
print("Loaded model from disk")

In [ ]:
print("Printing layers of Model :")
for layer in Xception_Model.layers:
    print(layer)

print()
print("Feature extraction from the model:")
feature_layer = Xception_Model.get_layer('xception')
print(feature_layer)

In [ ]:
xcep_feature_model = keras.Model(inputs = feature_layer.inputs, outputs = feature_layer.outputs)

In [ ]:
print("Xception Model Architecture")
keras.utils.plot_model(xcep_feature_model,'Inception_Model.png',show_shapes=True, show_layer_names=False, rankdir='TB', expand_nested=True, dpi=75)